---
title: Homework 2
author: "Group 5: Yilin Yang, Huiting Song, Kangheng Liu"
format:
  html:
    toc: true
    numbered: true
    toc-depth: 4
    self-contained: true
---

Please use jupyter notebook to implement the following structures. You will upload a Jupyter Notebook and an HTML or a PDF file (exported from the notebook) to the canvas portal. There is no restriction on what IDE to use, but a cloud IDE like Google Collab or AWS Sagemaker would be your best choice if you have a low-power/configuration computer. All the assignments are graded holistically. All assignments in the class are group assignments (if you prefer, you can choose to work independently).

This assignment will be graded out of 75 points. Each assignment submission page lists assignment open and close dates with grace periods (48 hours). They are also listed on the syllabus.

Since Blockchain is open-sourced, you will find most of the code online with a complete solution to most assignments. Please refrain from copy-pasting. The goal is to learn to build one from scratch to understand the technology. I will keep an eye on the submissions. Please list the source after the code to avoid plagiarism and copying issues.

The entire assignment is recorded and posted on Canvas under Panopto. You should go over the video and modify the code accordingly.

### 1       Importing data and packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from requests import Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects, HTTPError
import json
import os 
import seaborn as sns 
from datetime import datetime


In [2]:
plt.style.use('bmh')

### 2       Gather Coin token ID information from Coinmarket, Binance, or Coinbase.
You will need to search on any of the three websites to get the token IDs. The video shows how to do it through Binance and coin market cap. A slight difference between the Binance token and the coin market token is that the Binance token adds BUSD or USDT at the end, while the CoinMarket token does not. For example, Bitcoin is represented as BTC in CoinMarket and BTCBUSD or BTCUSDT in Binance. This represents the currency conversion to a stablecoin instead of a fiat currency.

For the sake of this bit, you can choose any 30 cryptocurrencies except Bitcoin and Ethereum. Here are the categories to select from

High market cap group - this will include the most widely traded coins. We will use the top 5 coins, namely, Bitcoin (BTC-USD), Ethereum (ETH-USD), Litecoin (LTC-USD), Teather (USDT), and BNB (BNB-USD).
Mid-market cap group will include coins with a market cap between 1 billion and 20 billion. We will use the top 5 coins, namely, Cardano (ADA-USD), Polygon (MATIC-USD), Dogecoin (DOGE-USD), Ripple (XRP-USD), and Binance USD (BUSD).
The low market cap group will include coins with a market cap between 100 million and 1 billion. We will use the top 20 coins.
Make sure to list the ticker ids in a list of strings

#### 2.1        Getting Data From Binance
Create a function that takes ticker interval, start_time, and end_time as input. We want to get all the data for the selected tickers and save them in data chosen frames. There is a way to automate this by using a function as well.

For each coin/token, you will get a dataframe of daily movements from start to end. The date and time are set as the index. Close, Symbol, and index are the most critical columns, and we want to subset them in a" coins" dataframe for all the coins.
Rename close to "Coinname_closeprice." Replace the coin name with the name of the coin and token id.

In [3]:
import requests

# Fetch the list of cryptocurrencies
url = 'https://api.coingecko.com/api/v3/coins/markets'
params = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 100,
    'page': 1,
}

response = requests.get(url, params=params)
coins = response.json()

# Sort the coins by market capitalization in descending order
coins.sort(key=lambda x: x['market_cap'], reverse=True)

# Define the groups
high_market_cap_group = []
mid_market_cap_group = []
low_market_cap_group = []

# Iterate through the coins and categorize them
for coin in coins:
    ticker_id = coin['id']
    market_cap = coin['market_cap']
    
    
    # High Market Cap Group
    if len(high_market_cap_group) < 5 and market_cap >= 20_000_000_000:
        high_market_cap_group.append(ticker_id)
        
    # Mid Market Cap Group
    elif len(mid_market_cap_group) < 5 and 1_000_000_000 <= market_cap < 20_000_000_000:
        mid_market_cap_group.append(ticker_id)
    
    # Low Market Cap Group
    elif len(low_market_cap_group) < 20 and 100_000_000 <= market_cap < 1_000_000_000:
        low_market_cap_group.append(ticker_id)

# Print the resulting groups
print("High Market Cap Group:", high_market_cap_group)
print("Mid Market Cap Group:", mid_market_cap_group)
print("Low Market Cap Group:", low_market_cap_group)




High Market Cap Group: ['bitcoin', 'ethereum', 'tether', 'binancecoin', 'ripple']
Mid Market Cap Group: ['staked-ether', 'solana', 'cardano', 'dogecoin', 'tron']
Low Market Cap Group: ['rocket-pool-eth', 'the-graph', 'algorand', 'whitebit', 'usdd', 'blockstack', 'immutable-x', 'xdce-crowd-sale', 'havven', 'elrond-erd-2', 'frax', 'eos', 'theta-token', 'injective-protocol', 'tezos', 'the-sandbox', 'bitget-token', 'axie-infinity', 'bitcoin-cash-sv', 'radix']


#### 2.2        Combining the dataframe
The updated dataframe should have the index and close price of all the coins of interest. We will use this to calculate the efficient frontier for the portfolio.

In [4]:
import requests
import pandas as pd
from datetime import datetime
import time

# Your predefined groups and codes remain unchanged

# Define the function to fetch historical data
def fetch_historical_data(coin_id, from_timestamp, to_timestamp, vs_currency='usd'):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range'
    attempts = 0
    max_attempts = 5
    wait_time = 10
    
    while attempts < max_attempts:
        response = requests.get(url, params={'vs_currency': vs_currency, 'from': from_timestamp, 'to': to_timestamp})
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            print(f"Hit rate limit fetching data for {coin_id}, retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            attempts += 1
            wait_time *= 2
        else:
            print(f"Failed to retrieve data for {coin_id}. Error code: {response.status_code}")
            return None
    
    print(f"Failed to retrieve data for {coin_id} after {max_attempts} attempts.")
    return None

# Get the Unix timestamp for 2022-09-27
from_timestamp = int(datetime.strptime('2022-09-27', '%Y-%m-%d').timestamp())

# Get the Unix timestamp for 2023-09-27
to_timestamp = int(datetime.strptime('2023-09-27', '%Y-%m-%d').timestamp())


# Adjusting the wait time between API calls to avoid rate limiting
time_between_calls = 60  # seconds

high_historical_data = {}

#Generate high market cap group first
for coin_id in high_market_cap_group:
    time.sleep(time_between_calls)  # adding sleep to avoid hitting rate limit
    
    data = fetch_historical_data(coin_id, from_timestamp, to_timestamp)
    if data and 'prices' in data and data['prices']:
        # Continue only if there is valid data
        df = pd.DataFrame(data['prices'], columns=['timestamp', f'{coin_id}_closeprice'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        high_historical_data[coin_id] = df
    else:
        print(f"No valid data available for {coin_id} in the given date range.")


# Combining all coin dataframes into one
if high_historical_data:
    coins_df = pd.concat(high_historical_data.values(), axis=1)
    print(coins_df.head())  # Print the first 5 rows of the resultant dataframe
    coins_df.to_csv('coin_prices.csv', index=True)
else:
    print("No valid historical data available for any coin in the given date range.")


# Combining all coin dataframes into one
high_coins_df = pd.concat(high_historical_data.values(), axis=1)

print(high_coins_df.head())  # Print the first 5 rows of the resultant dataframe


            bitcoin_closeprice  ethereum_closeprice  tether_closeprice  \
timestamp                                                                
2022-09-28        19116.331161          1331.150177           1.000675   
2022-09-29        19444.787771          1340.023059           1.001686   
2022-09-30        19563.765162          1335.738971           0.999983   
2022-10-01        19476.926596          1329.146032           1.000868   
2022-10-02        19314.463835          1311.263324           1.000526   

            binancecoin_closeprice  ripple_closeprice  
timestamp                                              
2022-09-28              272.550204           0.449739  
2022-09-29              281.358161           0.452320  
2022-09-30              283.748723           0.485497  
2022-10-01              284.007432           0.480186  
2022-10-02              282.869812           0.474961  
            bitcoin_closeprice  ethereum_closeprice  tether_closeprice  \
timestamp      

In [5]:
# Generate mid market cap group
mid_historical_data = {}

for coin_id in mid_market_cap_group:
    time.sleep(time_between_calls)  # adding sleep to avoid hitting rate limit
    
    data = fetch_historical_data(coin_id, from_timestamp, to_timestamp)
    if data and 'prices' in data and data['prices']:
        # Continue only if there is valid data
        df = pd.DataFrame(data['prices'], columns=['timestamp', f'{coin_id}_closeprice'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        mid_historical_data[coin_id] = df
    else:
        print(f"No valid data available for {coin_id} in the given date range.")


# Combining all coin dataframes into one
if mid_historical_data:
    coins_df = pd.concat(mid_historical_data.values(), axis=1)
    print(coins_df.head())  # Print the first 5 rows of the resultant dataframe
    coins_df.to_csv('coin_prices.csv', index=True)
else:
    print("No valid historical data available for any coin in the given date range.")


# Combining all coin dataframes into one
mid_coins_df = pd.concat(mid_historical_data.values(), axis=1)

print(mid_coins_df.head())  # Print the first 5 rows of the resultant dataframe

            staked-ether_closeprice  solana_closeprice  cardano_closeprice  \
timestamp                                                                    
2022-09-28              1327.688712          32.766122            0.442535   
2022-09-29              1335.960919          33.303300            0.438446   
2022-09-30              1330.090885          33.935996            0.437977   
2022-10-01              1326.804171          33.249664            0.434426   
2022-10-02              1308.867529          32.455791            0.430908   

            dogecoin_closeprice  tron_closeprice  
timestamp                                         
2022-09-28             0.060660         0.059472  
2022-09-29             0.060824         0.059755  
2022-09-30             0.060749         0.061011  
2022-10-01             0.061720         0.061151  
2022-10-02             0.060680         0.060523  
            staked-ether_closeprice  solana_closeprice  cardano_closeprice  \
timestamp         

In [6]:
# Generate low market cap group
low_historical_data = {}

for coin_id in low_market_cap_group:
    time.sleep(time_between_calls)  # adding sleep to avoid hitting rate limit
    
    data = fetch_historical_data(coin_id, from_timestamp, to_timestamp)
    if data and 'prices' in data and data['prices']:
        # Continue only if there is valid data
        df = pd.DataFrame(data['prices'], columns=['timestamp', f'{coin_id}_closeprice'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        low_historical_data[coin_id] = df
    else:
        print(f"No valid data available for {coin_id} in the given date range.")


# Combining all coin dataframes into one
if low_historical_data:
    coins_df = pd.concat(low_historical_data.values(), axis=1)
    print(coins_df.head())  # Print the first 5 rows of the resultant dataframe
    coins_df.to_csv('coin_prices.csv', index=True)
else:
    print("No valid historical data available for any coin in the given date range.")


# Combining all coin dataframes into one
low_coins_df = pd.concat(low_historical_data.values(), axis=1)

print(low_coins_df.head())  # Print the first 5 rows of the resultant dataframe

            rocket-pool-eth_closeprice  the-graph_closeprice  \
timestamp                                                      
2022-09-28                 1383.803801              0.099724   
2022-09-29                 1393.530770              0.099169   
2022-09-30                 1382.906860              0.099304   
2022-10-01                 1382.109407              0.098991   
2022-10-02                 1364.880039              0.098027   

            algorand_closeprice  whitebit_closeprice  usdd_closeprice  \
timestamp                                                               
2022-09-28             0.354013             4.053502         1.001474   
2022-09-29             0.350463             5.878442         1.002629   
2022-09-30             0.355148             5.879988         1.003861   
2022-10-01             0.353054            10.382280         1.001534   
2022-10-02             0.355094            10.780270         0.999831   

            blockstack_closeprice  immu

In [8]:
# Combining all three dataframes into one
combined_df = pd.concat([high_coins_df, mid_coins_df, low_coins_df], axis=1)

# Printing the first 5 rows of the resultant dataframe
print(combined_df.head())

# Save the combined dataframe to a CSV file
combined_df.to_csv('combined_coin_prices.csv', index=True)

            bitcoin_closeprice  ethereum_closeprice  tether_closeprice  \
timestamp                                                                
2022-09-28        19116.331161          1331.150177           1.000675   
2022-09-29        19444.787771          1340.023059           1.001686   
2022-09-30        19563.765162          1335.738971           0.999983   
2022-10-01        19476.926596          1329.146032           1.000868   
2022-10-02        19314.463835          1311.263324           1.000526   

            binancecoin_closeprice  ripple_closeprice  \
timestamp                                               
2022-09-28              272.550204           0.449739   
2022-09-29              281.358161           0.452320   
2022-09-30              283.748723           0.485497   
2022-10-01              284.007432           0.480186   
2022-10-02              282.869812           0.474961   

            staked-ether_closeprice  solana_closeprice  cardano_closeprice  \
tim